在程式碼的世界中  
重複的程式碼被視為邪惡的  
不應該有多個相同或類似的程式碼存在於不同地方  

這邊討論最著名的物件導向原則：繼承  
繼承可以讓我們可以在兩個或以上的類別之間建構〝是一個〞的關係  
這邊會討論：  
* 基本繼承
* 從內建繼承
* 多重繼承
* 多型與鴨子型別

# 基本繼承
---

基本上  
每個類別都使用了繼承，都是繼承object這個特殊類別的子類別  
此類別提供了一些資料與行為(都是內部使用的雙底線行為)  
若我們沒有明確的繼承其他類別  
Python則會自動繼承object

In [2]:
class MySubClass(object):
    pass

父類別、超類別是指繼承來源  
子類別則是繼承者  
上述例子中  
父類別是object  
子類別是MySubClass  
子類別是從父類別衍伸出來，或者說擴充的父輩  

我們先從一個聯絡人管理系統開始  
它可以記錄多個人的姓名與電子郵件  
聯絡人類別負責在類別變數中維護聯絡人清單並初始化個人聯絡人的姓名與地址

In [21]:
class Contact:
    all_contacts = []
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)
        
    def __repr__(self) -> str:
        # __repr__ !r 是把回傳的文字加上 ''
        return(f"{self.__class__.__name__}("
               f"{self.name!r}, {self.email!r}"
               f")"
        )

> 小心這個語法，若曾經使用過self.all_contacts設定變數，  
> 則實際上會建構只與該類別物件關聯的新變數實例，  
> 透過Contact.all_contacts存取的類別變數還是不變

建構一個新的Supplier類別，  
其動作與Contact類別相似，  
但Supplier多了自己的order方法

In [22]:
class Supplier(Contact):
    def order(self, order):
        print(f"If this were a real system we would send '{order}' to '{self.name}'")

In [23]:
c = Contact("Some body", "somebody@example.net")
s = Supplier("Sup Plier","supplier@example.net")
print(c.name, c.email, s.name, s.email)

Some body somebody@example.net Sup Plier supplier@example.net


In [27]:
c.all_contacts

[Contact('Some body', 'somebody@example.net'),
 Supplier('Sup Plier', 'supplier@example.net')]

In [29]:
c_1 = Contact('Dusty', 'dusty@example.com')
c_2 = Contact('Steve', 'steve@example.com')

from pprint import pprint
pprint(c.all_contacts)

[Contact('Some body', 'somebody@example.net'),
 Supplier('Sup Plier', 'supplier@example.net'),
 Contact('Dusty', 'dusty@example.com'),
 Contact('Steve', 'steve@example.com')]


In [7]:
c.all_contacts[0].name

'Some body'

In [8]:
c.order("I need Pliers")

AttributeError: 'Contact' object has no attribute 'order'

In [9]:
s.order("I need Pliers")

If this were a real system we would send 'I need Pliers' to 'Sup Plier'


# 擴充內建
---

若我們要以名稱搜尋清單  
我們可以加入方法到Contact中來搜尋

In [30]:
class ContactList(list):
    def search(self, name):
        matching_contacts = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts

In [31]:
class Contact:
    all_contacts = ContactList()
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        self.all_contacts.append(self)

In [32]:
c1 = Contact('John A', 'johna@example.net')
c2 = Contact('John B', 'johnb@example.net')
c3 = Contact('Jenna C', 'jennac@example.net')
[c.name for c in Contact.all_contacts.search('John')]

['John A', 'John B']

In [33]:
c1.all_contacts

In [34]:
Contact.all_contacts

以上是擴充list類別  
我們接下來擴充一個dict類別  
在dict上增加一個找出最長key的方法  

In [35]:
class LongNameDict(dict):
    def longest_key(self):
        '''
        找出LongNameDict中
        名字最長的key
        '''
        longest = None
        for key in self:
            if not longest or len(key) > len(longest):
                longest = key
        return longest

In [37]:
longkeys = LongNameDict()
longkeys['hello'] = 1
longkeys['longest yet'] = 5
longkeys['hello2'] = 'world'
longkeys.longest_key()

'longest yet'

In [38]:
max(longkeys, key=len)

'longest yet'

# 複寫與super
---